In [ ]:
import torch/
        self.signals_df = signals_df
        self.meta_df = meta_df
        self.lead = lead
        self.seq_len = seq_len
        self.signals_map = dict()
        for ecgid in self.ecgids:
            signals = self.signals_df[self.signals_df['ecg_id'] == ecgid]
            if signals.empty:
                continue
            lead_col = f'channel-{ord(lead) - ord("I")}'
            signal_arr = signals[lead_col].values
            if len(signal_arr) < seq_len:
                pad = np.zeros(seq_len)
                pad[:len(signal_arr)] = signal_arr
                signal_arr = pad
            elif len(signal_arr) > seq_len:
                signal_arr = signal_arr[:seq_len]
            self.signals_map[ecgid] = signal_arr.astype(np.float32)
        self.labels_map = meta_df.set_index('ecg_id')['risk_label'].to_dict()
        self.valid_ecgids = [eid for eid in self.ecgids if eid in self.signals_map and self.labels_map.get(eid, -1) >= 0]

    def __len__(self):
        return len(self.valid_ecgids)

    def __getitem__(self, idx):
        ecgid = self.valid_ecgids[idx]
        signal = self.signals_map[ecgid]
        label = self.labels_map[ecgid]
        return torch.tensor(signal).unsqueeze(0), torch.tensor(label).long()

# CNN model for raw ECG inputs
class DeepECGClassifier(nn.Module):
    def __init__(self, seq_len=5000, num_classes=5):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 32, 15, padding=7)
        self.bn1 = nn.BatchNorm1d(32)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(32, 64, 15, padding=7)
        self.bn2 = nn.BatchNorm1d(64)
        self.pool2 = nn.MaxPool1d(2)
        self.conv3 = nn.Conv1d(64, 128, 15, padding=7)
        self.bn3 = nn.BatchNorm1d(128)
        self.pool3 = nn.MaxPool1d(2)
        self.conv4 = nn.Conv1d(128, 256, 15, padding=7)
        self.bn4 = nn.BatchNorm1d(256)
        self.pool4 = nn.MaxPool1d(2)
        def convpool_out_length(l_in, kernel=15, padding=7, pool=2, layers=4):
            l = l_in
            for _ in range(layers):
                l = (l + 2*padding - kernel + 1)
                l = l // pool
            return l
        out_len = convpool_out_length(seq_len)
        self.fc1 = nn.Linear(256 * out_len, 128)
        self.dropout = nn.Dropout(0.4)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = self.pool4(F.relu(self.bn4(self.conv4(x))))
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

#yes, I daksh vohra used fstrings
train_signals = pd.read_csv(f'{path}/train_signal.csv')
train_meta = pd.read_csv(f'{path}/train_meta.csv')
valid_signals = pd.read_csv(f'{path}/valid_signal.csv')
valid_meta = pd.read_csv(f'{path}/valid_meta.csv')
test_signals = pd.read_csv(f'{path}/test_signal.csv')
test_meta = pd.read_csv(f'{path}/test_meta.csv')


train_dataset = PTBXLRawECGDataset(train_signals, train_meta, lead='I', seq_len=5000)
valid_dataset = PTBXLRawECGDataset(valid_signals, valid_meta, lead='I', seq_len=5000)
test_dataset = PTBXLRawECGDataset(test_signals, test_meta, lead='I', seq_len=5000)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DeepECGClassifier(seq_len=5000, num_classes=5).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

#useful later
def evaluate(loader):
    model.eval()
    total_loss, total_correct, total = 0, 0, 0
    with torch.no_grad():
        for signals, labels in loader:
            signals, labels = signals.to(device), labels.to(device)
            outputs = model(signals)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * labels.size(0)
            predicted = outputs.argmax(dim=1)
            total_correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return total_loss / total if total > 0 else 0, total_correct / total if total > 0 else 0

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss, total_correct, total = 0, 0, 0
    for signals, labels in train_loader:
        signals, labels = signals.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(signals)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * labels.size(0)
        predicted = outputs.argmax(dim=1)
        total_correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_loss = total_loss / total
    train_acc = total_correct / total
    val_loss, val_acc = evaluate(valid_loader)

    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss {train_loss:.4f}, Train Acc {train_acc:.4f}, Val Loss {val_loss:.4f}, Val Acc {val_acc:.4f}")

test_loss, test_acc = evaluate(test_loader)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

Using Colab cache for faster access to the 'ptb-xl-dataset-reformatted' dataset.
Path to dataset files: /kaggle/input/ptb-xl-dataset-reformatted


KeyError: 'risk_label'

In [ ]:
print(train_meta.columns)
print(valid_meta.columns)
print(test_meta.columns)